In [58]:
#instalando java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [59]:
#instalando spark
!pip install pyspark

In [60]:
#configurando variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"

In [61]:
#iniciando SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext

spark = SparkSession.builder.master("local").appName("Exercício Intro").getOrCreate()


#Etapa 1

In [112]:
#carregando o arquivo
caminho = '/content/nomes_aleatorios.txt'

df_nomes = spark.read.csv(caminho)

df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



#Etapa 2

In [113]:
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")
df_nomes.printSchema()
df_nomes.show(10)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



#Etapa 3

In [114]:
from pyspark.sql.functions import when, rand

df_nomes = df_nomes.withColumn(
    "Escolaridade",
    when(rand() < 0.33, "Fundamental").otherwise(
        when(rand() < 0.66, "Medio").otherwise("Superior")
    ),
)

df_nomes.show(10)

+-----------------+------------+
|            Nomes|Escolaridade|
+-----------------+------------+
|   Frances Bennet|    Superior|
|    Jamie Russell|       Medio|
|   Edward Kistler|       Medio|
|    Sheila Maurer|       Medio|
| Donald Golightly|       Medio|
|       David Gray| Fundamental|
|      Joy Bennett| Fundamental|
|      Paul Kriese| Fundamental|
|Berniece Ornellas|       Medio|
|    Brian Farrell| Fundamental|
+-----------------+------------+
only showing top 10 rows



#Etapa 4

In [115]:
paises = ["Itália",
          "Holanda",
          "Espanha",
          "Omã",
          "Sri Lanka",
          "Grécia",
          "Brasil",
          "Austrália",
          "Nova Zelândia",
          "Noruega",
          "Suíça",
          "Japão",
          "Egito"]

df_nomes = df_nomes.withColumn(
    "Pais",
    when(rand() < 1 / 13, paises[0])
    .when(rand() < 2 / 13, paises[1])
    .when(rand() < 3 / 13, paises[2])
    .when(rand() < 4 / 13, paises[3])
    .when(rand() < 5 / 13, paises[4])
    .when(rand() < 6 / 13, paises[5])
    .when(rand() < 7 / 13, paises[6])
    .when(rand() < 8 / 13, paises[7])
    .when(rand() < 9 / 13, paises[8])
    .when(rand() < 10 / 13, paises[9])
    .when(rand() < 11 / 13, paises[10])
    .when(rand() < 12 / 13, paises[11])
    .otherwise(paises[12]),
)

df_nomes.show(10)

+-----------------+------------+---------+
|            Nomes|Escolaridade|     Pais|
+-----------------+------------+---------+
|   Frances Bennet|    Superior|   Grécia|
|    Jamie Russell|       Medio|   Grécia|
|   Edward Kistler|       Medio|Sri Lanka|
|    Sheila Maurer|       Medio|  Holanda|
| Donald Golightly|       Medio|  Espanha|
|       David Gray| Fundamental|  Espanha|
|      Joy Bennett| Fundamental|   Brasil|
|      Paul Kriese| Fundamental|   Itália|
|Berniece Ornellas|       Medio|   Grécia|
|    Brian Farrell| Fundamental|  Espanha|
+-----------------+------------+---------+
only showing top 10 rows



#Etapa 5

In [116]:
from pyspark.sql.functions import floor

df_nomes = df_nomes.withColumn(
    "AnoNascimento", floor(rand() * (2010 - 1945 + 1) + 1945)
)

df_nomes.show(10)

+-----------------+------------+---------+-------------+
|            Nomes|Escolaridade|     Pais|AnoNascimento|
+-----------------+------------+---------+-------------+
|   Frances Bennet|    Superior|   Grécia|         1980|
|    Jamie Russell|       Medio|   Grécia|         1962|
|   Edward Kistler|       Medio|Sri Lanka|         1951|
|    Sheila Maurer|       Medio|  Holanda|         1951|
| Donald Golightly|       Medio|  Espanha|         1967|
|       David Gray| Fundamental|  Espanha|         1954|
|      Joy Bennett| Fundamental|   Brasil|         1994|
|      Paul Kriese| Fundamental|   Itália|         1991|
|Berniece Ornellas|       Medio|   Grécia|         1964|
|    Brian Farrell| Fundamental|  Espanha|         1972|
+-----------------+------------+---------+-------------+
only showing top 10 rows



#Etapa 6

In [102]:
df_select = df_nomes.filter(df_nomes.AnoNascimento >= 2001)

df_select.show(10)

+---------------+------------+---------+-------------+
|          Nomes|Escolaridade|     Pais|AnoNascimento|
+---------------+------------+---------+-------------+
|   David Medina| Fundamental|Sri Lanka|         2003|
| Lorenzo Woodis|       Medio|  Holanda|         2001|
|    Albert Leef|       Medio|      Omã|         2001|
|Helen Blackwell| Fundamental|Sri Lanka|         2006|
|   Rebecca Snow|    Superior|Sri Lanka|         2010|
| Amanda Gravitt|       Medio|      Omã|         2010|
|   Roxie Bernal|       Medio|   Grécia|         2001|
|   James Barton| Fundamental|   Itália|         2005|
|  George Miller|       Medio|  Espanha|         2004|
|   Juliet Liles|       Medio|  Holanda|         2002|
+---------------+------------+---------+-------------+
only showing top 10 rows



#Etapa 7

In [103]:
df_select.createOrReplaceTempView("pessoas")

spark.sql("SELECT * FROM pessoas WHERE AnoNascimento >= 2001").show(10)

+---------------+------------+---------+-------------+
|          Nomes|Escolaridade|     Pais|AnoNascimento|
+---------------+------------+---------+-------------+
|   David Medina| Fundamental|Sri Lanka|         2003|
| Lorenzo Woodis|       Medio|  Holanda|         2001|
|    Albert Leef|       Medio|      Omã|         2001|
|Helen Blackwell| Fundamental|Sri Lanka|         2006|
|   Rebecca Snow|    Superior|Sri Lanka|         2010|
| Amanda Gravitt|       Medio|      Omã|         2010|
|   Roxie Bernal|       Medio|   Grécia|         2001|
|   James Barton| Fundamental|   Itália|         2005|
|  George Miller|       Medio|  Espanha|         2004|
|   Juliet Liles|       Medio|  Holanda|         2002|
+---------------+------------+---------+-------------+
only showing top 10 rows



#Etapa 8

In [109]:
df_nomes = df_nomes.filter((df_nomes.AnoNascimento >= 1980) & (df_nomes.AnoNascimento <= 1994))
millenials = df_nomes.count()

print(millenials)

2286754


#Etapa 9

In [110]:
df_nomes.createOrReplaceTempView("pessoas")

spark.sql("SELECT COUNT(*) FROM pessoas WHERE AnoNascimento >= 1980 AND AnoNascimento <= 1994").show(10)

+--------+
|count(1)|
+--------+
| 2286754|
+--------+



#Etapa 10

In [117]:
df_nomes.createOrReplaceTempView("pessoas")

df_geracoes = spark.sql(
    """SELECT Pais,
          CASE
              WHEN
              AnoNascimento BETWEEN 1944 AND 1964
              THEN 'Baby Boomers'

              WHEN
              AnoNascimento BETWEEN 1965 AND 1979
              THEN 'Geração X'

              WHEN
              AnoNascimento BETWEEN 1980 AND 1994
              THEN 'Millenials'

              WHEN
              AnoNascimento BETWEEN 1995 AND 2015
              THEN 'Geração Z'

              END AS Geracao,

              COUNT (*) AS Quantidade
              FROM pessoas
              GROUP BY Pais, Geracao
              ORDER BY Pais ASC, Geracao ASC, Quantidade ASC

"""
)

df_geracoes.show()

+---------+------------+----------+
|     Pais|     Geracao|Quantidade|
+---------+------------+----------+
|Austrália|Baby Boomers|    119449|
|Austrália|   Geração X|     89943|
|Austrália|   Geração Z|     95957|
|Austrália|  Millenials|     89868|
|   Brasil|Baby Boomers|    226572|
|   Brasil|   Geração X|    170243|
|   Brasil|   Geração Z|    181011|
|   Brasil|  Millenials|    169845|
|    Egito|Baby Boomers|        77|
|    Egito|   Geração X|        57|
|    Egito|   Geração Z|        49|
|    Egito|  Millenials|        39|
|  Espanha|Baby Boomers|    550684|
|  Espanha|   Geração X|    412297|
|  Espanha|   Geração Z|    438735|
|  Espanha|  Millenials|    413496|
|   Grécia|Baby Boomers|    359900|
|   Grécia|   Geração X|    269723|
|   Grécia|   Geração Z|    288719|
|   Grécia|  Millenials|    270208|
+---------+------------+----------+
only showing top 20 rows

